In [0]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
"/content/drive/My Drive/DSCI 303 Final Project/Data_Trimming/preprocessed_150k_reviews.json"

'/content/drive/My Drive/DSCI 303 Final Project/Data_Trimming/preprocessed_150k_reviews.json'

In [0]:
root_path = 'drive/My Drive/DSCI 303 Final Project/Data Trimming/'

In [0]:
# with open(root_path + 'preprocessed_150k_reviews.json') as json_file2: # Open json file to load
with open ("/content/drive/My Drive/DSCI 303 Final Project/Data_Trimming/preprocessed_150k_reviews.json") as json_file2:
    data2 = json.load(json_file2)
    data3 = json.loads(data2)

In [0]:
df = pd.DataFrame.from_dict(data3)
X_raw = df['review']
Y_raw = df['stars']

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_raw)

In [0]:
# file_dump = open("vectorizer", 'wb')

# pickle.dump(vectorizer, file_dump)

In [9]:
X

<150000x67816 sparse matrix of type '<class 'numpy.int64'>'
	with 6869739 stored elements in Compressed Sparse Row format>

In [0]:
# scipy.sparse.save_npz("X",X)

In [0]:
Y = Y_raw

In [0]:
# np.save("Y",Y)

In [0]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
# from sklearn.model_selection import GridSearchCV

In [0]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=101)

In [0]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg_accs = []
for pen in ['l1', 'l2', 'elasticnet','none']:
  log_reg = LogisticRegression(penalty = pen, random_state=0, multi_class = 'multinomial', solver = 'saga')
  log_reg_scores = cross_val_score(log_reg, x_train, y_train, cv=10)
  print(log_reg_scores)
  log_reg_accs.append(log_reg_scores)



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [0]:
# Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB

In [0]:
# Cross validation for MNB

from sklearn.model_selection import cross_val_score
mnb = MultinomialNB()
mnb_cv_scores = cross_val_score(mnb, x_train, y_train, cv=10)
print(mnb_cv_scores)

[0.59756707 0.59840027 0.59631728 0.59773371 0.60075    0.59741667
 0.59383333 0.59134856 0.59431525 0.59998333]


In [0]:
# MBN Test set!
mnb = MultinomialNB()
mnb.fit(x_train,y_train)
predmnb = mnb.predict(x_test)
print("Confusion Matrix for Multinomial Naive Bayes:")
print(confusion_matrix(y_test,predmnb))
print("Score:",round(accuracy_score(y_test,predmnb)*100,2))
print("Classification Report:",classification_report(y_test,predmnb))

Confusion Matrix for Multinomial Naive Bayes:
[[3101  747  292  120  117]
 [ 656  556  721  317  108]
 [ 326  290 1028 1308  300]
 [ 255  104  510 3983 1831]
 [ 647   51  124 3275 9233]]
Score: 59.67
Classification Report:               precision    recall  f1-score   support

         1.0       0.62      0.71      0.66      4377
         2.0       0.32      0.24      0.27      2358
         3.0       0.38      0.32      0.35      3252
         4.0       0.44      0.60      0.51      6683
         5.0       0.80      0.69      0.74     13330

    accuracy                           0.60     30000
   macro avg       0.51      0.51      0.51     30000
weighted avg       0.61      0.60      0.60     30000



In [0]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
random_forest_cv_avg_accs = []

In [0]:
# Cross validation for Random Forest Classifier
for n_trees in np.arange(10,160,20):
  rfc = RandomForestClassifier(n_estimators = n_trees)
  rfc_cv_scores = cross_val_score(rfc, x_train, y_train, cv=8)
  print("CV with {} trees:".format(n_trees))
  print(rfc_cv_scores)
  random_forest_cv_avg_accs.append(np.mean(rfc_cv_scores))

CV with 10 trees:
[0.535031   0.54786667 0.5446     0.5446     0.54373333 0.5338
 0.53466667 0.54370291]
CV with 30 trees:
[0.57669489 0.57793333 0.57466667 0.57393333 0.5706     0.56953333
 0.5712     0.57357157]
CV with 50 trees:
[0.58016132 0.5836     0.5784     0.57726667 0.5752     0.57606667
 0.58       0.57983866]
CV with 70 trees:
[0.57856143 0.5808     0.57906667 0.58006667 0.5832     0.5788
 0.58553333 0.58110541]
CV with 90 trees:
[0.57876142 0.57926667 0.57993333 0.58293333 0.58013333 0.5814
 0.58166667 0.58417228]
CV with 110 trees:
[0.57969469 0.58406667 0.58073333 0.58186667 0.58246667 0.5792
 0.5822     0.58543903]
CV with 130 trees:
[0.58036131 0.5834     0.58226667 0.58266667 0.57953333 0.5792
 0.5818     0.58010534]


In [0]:
plt.plot(np.arange(10,160,20), random_forest_cv_avg_accs)
plt.title("Random Forest CV Avg Accuracy vs. number of trees")
plt.show()

In [17]:
# Test set!!!
chosen_n_trees = 110

rmfr = RandomForestClassifier(n_estimators = chosen_n_trees)
rmfr.fit(x_train,y_train)
predrmfr = rmfr.predict(x_test)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrmfr))
forest_acc = round(accuracy_score(y_test,predrmfr)*100,2)
print("Score:", forest_acc)
print("Classification Report:",classification_report(y_test,predrmfr))
random_forest_accs.append(forest_acc)

Confusion Matrix for Random Forest Classifier:
[[ 3116    18    47   136  1060]
 [  747    43   150   386  1032]
 [  278    18   211  1023  1722]
 [  110     4    65  1389  5115]
 [  121     2    10   462 12735]]
Score: 58.31
Classification Report:               precision    recall  f1-score   support

         1.0       0.71      0.71      0.71      4377
         2.0       0.51      0.02      0.04      2358
         3.0       0.44      0.06      0.11      3252
         4.0       0.41      0.21      0.28      6683
         5.0       0.59      0.96      0.73     13330

    accuracy                           0.58     30000
   macro avg       0.53      0.39      0.37     30000
weighted avg       0.54      0.58      0.50     30000



NameError: ignored

In [18]:
from sklearn.linear_model import LogisticRegression
for iter_num in np.arange(100,700,100):
  clf400 = LogisticRegression(random_state=0, max_iter = iter_num, solver='lbfgs', multi_class='multinomial')
  clf400.fit(x_train, y_train)
  predlog = clf400.predict(x_test)
  print("Confusion Matrix for Logistic Regression TFIDF:")
  print(confusion_matrix(y_test,predlog))
  print("Score:",round(accuracy_score(y_test,predlog)*100,2))
  print("Classification Report:",classification_report(y_test,predlog))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Confusion Matrix for Logistic Regression TFIDF:
[[ 3356   505   170   102   244]
 [  737   726   538   202   155]
 [  258   420  1179   933   462]
 [   80   144   686  2709  3064]
 [  157    65   204  1550 11354]]
Score: 64.41
Classification Report:               precision    recall  f1-score   support

         1.0       0.73      0.77      0.75      4377
         2.0       0.39      0.31      0.34      2358
         3.0       0.42      0.36      0.39      3252
         4.0       0.49      0.41      0.44      6683
         5.0       0.74      0.85      0.79     13330

    accuracy                           0.64     30000
   macro avg       0.56      0.54      0.54     30000
weighted avg       0.62      0.64      0.63     30000



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Confusion Matrix for Logistic Regression TFIDF:
[[ 3332   544   179    98   224]
 [  730   735   545   201   147]
 [  277   435  1201   899   440]
 [   85   160   689  2775  2974]
 [  160    73   219  1640 11238]]
Score: 64.27
Classification Report:               precision    recall  f1-score   support

         1.0       0.73      0.76      0.74      4377
         2.0       0.38      0.31      0.34      2358
         3.0       0.42      0.37      0.39      3252
         4.0       0.49      0.42      0.45      6683
         5.0       0.75      0.84      0.79     13330

    accuracy                           0.64     30000
   macro avg       0.55      0.54      0.54     30000
weighted avg       0.62      0.64      0.63     30000



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


...

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Confusion Matrix for Logistic Regression TFIDF:
[[ 3307   550   196   101   223]
 [  747   718   535   202   156]
 [  269   441  1203   896   443]
 [   84   160   680  2764  2995]
 [  164    60   208  1665 11233]]
Score: 64.08
Classification Report:               precision    recall  f1-score   support

         1.0       0.72      0.76      0.74      4377
         2.0       0.37      0.30      0.33      2358
         3.0       0.43      0.37      0.40      3252
         4.0       0.49      0.41      0.45      6683
         5.0       0.75      0.84      0.79     13330

    accuracy                           0.64     30000
   macro avg       0.55      0.54      0.54     30000
weighted avg       0.62      0.64      0.63     30000



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Confusion Matrix for Logistic Regression TFIDF:
[[ 3307   550   196    97   227]
 [  753   718   526   205   156]
 [  271   436  1204   905   436]
 [   85   159   685  2747  3007]
 [  166    61   212  1651 11240]]
Score: 64.05
Classification Report:               precision    recall  f1-score   support

         1.0       0.72      0.76      0.74      4377
         2.0       0.37      0.30      0.34      2358
         3.0       0.43      0.37      0.40      3252
         4.0       0.49      0.41      0.45      6683
         5.0       0.75      0.84      0.79     13330

    accuracy                           0.64     30000
   macro avg       0.55      0.54      0.54     30000
weighted avg       0.62      0.64      0.63     30000

Confusion Matrix for Logistic Regression TFIDF:
[[ 3299   560   194    97   227]
 [  752   713   532   205   156]
 [  272   433  1203   905   439]
 [   85   157   681  2754  3006]
 [  165    54   210  1661 11240]]
Score: 64.03
Classification Report:            

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [0]:
# K Nearest Neighbour Algorithm
from sklearn.neighbors import KNeighborsClassifier
knn_cv_avg_accs = []

In [0]:
# KNN CV

for k_n in np.arange(10,60,10):
  knn = KNeighborsClassifier(n_neighbors=k_n)
  knn_cv_scores = cross_val_score(knn, x_train, y_train, cv=8)
  print("CV with {} neighbors:".format(k_n))
  print(knn_cv_scores)
  knn_cv_avg_accs.append(np.mean(knn_cv_scores))

In [16]:
# KNN test set!
knn_optimal_k = 20

knn = KNeighborsClassifier(n_neighbors=knn_optimal_k )
knn.fit(x_train,y_train)
predknn = knn.predict(x_test)
print("Confusion Matrix for K Neighbors Classifier:")
print(confusion_matrix(y_test,predknn))
knn_acc = round(accuracy_score(y_test,predknn)*100,2)
print("Score: ",knn_acc)
print("Classification Report:")
print(classification_report(y_test,predknn))

Confusion Matrix for K Neighbors Classifier:
[[ 1468    94    37   180  2598]
 [  329    69    72   307  1581]
 [  179    84   107   669  2213]
 [  120    70    69  1029  5395]
 [  115    72    18   514 12611]]
Score:  50.95
Classification Report:
              precision    recall  f1-score   support

         1.0       0.66      0.34      0.45      4377
         2.0       0.18      0.03      0.05      2358
         3.0       0.35      0.03      0.06      3252
         4.0       0.38      0.15      0.22      6683
         5.0       0.52      0.95      0.67     13330

    accuracy                           0.51     30000
   macro avg       0.42      0.30      0.29     30000
weighted avg       0.46      0.51      0.42     30000



In [0]:
# file_read = open('vectorizer', 'rb')
# hi = pickle.load(file_read)

In [0]:
hi

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)